In [11]:
import pandas as pd
import numpy as np
import os
import numpy
import matplotlib.pyplot as plt
import SimpleITK
import itertools
import sys
import torch
from torchvision import transforms
from PIL import Image
from matplotlib import cm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from pathlib import Path

SOURCE_PATH = Path(os.getcwd()) / 'src'

if SOURCE_PATH not in sys.path:
    sys.path.append(SOURCE_PATH)

from src.extraction import (
    get_images_lists_from_path,
    get_images_lists_from_more_paths
)

from src.plots import (
    plot_observation
)

from src.loading import (
    load_images_from_paths
)

from src.modelling import (
    deeplab_finetuning
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
path = Path(os.getcwd()) / 'data_extracted'
type_to_use = 'flair'

deeplab_finetuning(
    path, 
    type_to_use, 
    'seg', 
    'models', 
    model_exp_name='flair_model1_noshuffle',
    pretrained=False,
    epochs=2, 
    batch_size=16)

Model Created
Parameters defined
Dataloaders created
Epoch 1/2
----------


100%|██████████| 4/4 [07:28<00:00, 112.24s/it]


Train Loss: 0.0888


100%|██████████| 1/1 [00:31<00:00, 31.15s/it]


Test Loss: 0.0056
{'epoch': 1, 'Train_loss': 0.08882245421409607, 'Test_loss': 0.005575497634708881, 'Train_f1_score': 0.04317088404917598, 'Test_f1_score': 0.0}
Epoch 2/2
----------


100%|██████████| 4/4 [08:12<00:00, 123.01s/it]


Train Loss: 0.0599


100%|██████████| 1/1 [00:28<00:00, 28.28s/it]


Test Loss: 0.0081
{'epoch': 2, 'Train_loss': 0.059866614639759064, 'Test_loss': 0.008092830888926983, 'Train_f1_score': 0.04636563595656967, 'Test_f1_score': 0.0}
Training complete in 16m 41s
Lowest Loss: 0.005575


In [13]:
# path = Path(os.getcwd()) / 'data_extracted'
# type_to_use = 'flair'

# deeplab_finetuning(
#     path, 
#     type_to_use, 
#     'seg', 
#     'models', 
#     model_exp_name='flair_model1_pretrained', 
#     epochs=2, 
#     batch_size=16,
#     pretrained=True)

In [15]:
modelname = 'flair_model1_noshuffle.pt'
model = torch.load(Path(os.getcwd()) / 'models' / modelname)

In [ ]:
model

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se